In [ ]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
    'minScale': 1, # May create problems
    'width': 1024,# * .75,
    'height': 900,# * .75,
});


%matplotlib inline
import seaborn as sns
import numpy as np
from time import sleep
from IPython.display import clear_output

sns.set(rc={'figure.figsize': (15, 6)})
np.set_printoptions(precision=5, threshold=100)

# Why PyEPICS?

## Because Python is awesome!

![https://xkcd.com/353/](images/xkcd-800.png)

* Easy to learn
* Highly readable syntax
* Powerful features

### PyEPICS enables you to...

* Quickly access data from EPICS PVs in convenient formats (eg numpy arrays)
* Easily create Python representations of physical devices
* Automate interactions with multiple devices
* Avoid the low-level details of the channel access protocol and focus on your application

![PyEPICS](images/pyepics.png)

### Dependencies

* [EPICS Base](http://www.aps.anl.gov/epics/base/R3-14/12.php) libraries
* Python 2.6+ or 3.2+... ***2.7 or 3.4+ highly recommended***
* *Optional:* numpy for nicer array handling

### Installation

In your terminal:

```bash
pip3 install pyepics
```

Set the environment variables (eg in your `~/.bashrc` file):

```bash
export EPICS_BASE=/epics/base
export EPICS_HOST_ARCH=linux-x86_64
```

# Let's get some PVs

In [ ]:
import epics

In [ ]:
temperature_pv = epics.PV('PYEPICS_DEMO:TEMPERATURE')

In [ ]:
temperature_pv.get()

In [ ]:
temperature_pv.value

# Stringy PVs

## Enums

In [ ]:
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR')

colour_pv.get()

In [ ]:
colour_pv.get(as_string=True)

In [ ]:
colour_pv.char_value

# Arrays and waveforms

In [ ]:
wave_pv = epics.PV('PYEPICS_DEMO:WAVE')

data = wave_pv.get()

data

In [ ]:
import matplotlib.pyplot as plt

plt.plot(data)

## What happens if a PV doesn't exist

In [ ]:
invalid_pv = epics.PV('INVALID_NAME')

print(invalid_pv.get())

## Lesson

Don't do this:

```python
if not binary_pv.value:
    
    do_something()
```

Explicity check the value:

```python
if binary_pv.value == 0:
    
    do_something()
```

# Setting Values

In [ ]:
setpoint_pv = epics.PV('PYEPICS_DEMO:SETPOINT')

In [ ]:
setpoint_pv.put(5.2)

In [ ]:
setpoint_pv.value = 2.3

### Enum PVs accept int or str

In [ ]:
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR')

In [ ]:
colour_pv.put(1)

In [ ]:
colour_pv.put('red')


## Put doesn't guarantee the value was set

### When outside of drive limits

In [ ]:
setpoint_pv.put(15)  # PYEPICS_DEMO:SETPOINT has a DRVH of 10

In [ ]:
setpoint_pv.value

### When the PV is disconnected

In [ ]:
disconnected_pv = epics.PV('INVALID_NAME')

In [ ]:
disconnected_pv.put(3)

# Callback functions

## Connection callbacks

In [ ]:
def on_connection_change(pvname, conn, pv):
    print(pvname, 'connected:', conn, flush=True)

In [ ]:
colour_pv = epics.PV('PYEPICS_DEMO:COLOUR',
                     connection_callback=on_connection_change)

## Value callbacks

In [ ]:
def on_value(pvname, value, char_value, timestamp, **kwargs):
    print(pvname, timestamp, value, char_value, flush=True)
    print('severity:', kwargs['severity'],
          'status:', kwargs['status'], flush=True)

#### Add the callback to the PV

In [ ]:
callback_id = colour_pv.add_callback(on_value)

#### Remove the callback

In [ ]:
colour_pv.remove_callback(callback_id)

#### Remove all callbacks

In [ ]:
colour_pv.clear_callbacks()

### What *not* to do inside callback functions

Any other channel access function calls

Slow / resource intensive processing; the callback should complete in ms

In [171]:
def temperature_callback(value, **kwargs):
    clear_output()
    print('temperature:', value, flush=True)
    
def colour_callback(char_value, **kwargs):
    print('doing something slow in colour_callback', flush=True)
    time.sleep(4.)

temperature_pv.add_callback(temperature_callback)
colour_pv.add_callback(colour_callback)

temperature: 31.740669313818216


In [172]:
temperature_pv.clear_callbacks()
colour_pv.clear_callbacks()

### Options

Spawn a new thread from the callback

Trigger processing on a worker thread using a queue or event 

## Deferring to a worker thread

In [182]:
from threading import Event, Thread
colour_change = Event()

def temperature_callback(value, **kwargs):
    clear_output()
    print('temperature:', value, flush=True)
    
def colour_callback(char_value, **kwargs):
    colour_change.set()
    
def worker():
    while True:
        colour_change.wait()
        print('**** STARTING SLOW PROCESS ****', flush=True)
        time.sleep(4.)
        print('************ DONE *************', flush=True)
        colour_change.clear()

worker_thread = Thread(target=worker)
worker_thread.start()
        
temperature_pv.add_callback(temperature_callback)
colour_pv.add_callback(colour_callback)

temperature: 31.967488523572683


In [183]:
temperature_pv.clear_callbacks()
colour_pv.clear_callbacks()